This is the code and procedure if you want to predict with trained XGBOOST models on new data

In [1]:
!pip install rdkit-pypi
!pip install mordred

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 29.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 20.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mordred: filename=mordred-1.2.0-py3-none-any.whl size=176723 sha256=a7641ad3a3bc8561fd824f0c13de9c2482f036439b82008f27855476a36aba32
  Stored in directory: /root/.cache/pip/wheels/05/95/d1/9e913738f0e8362b3676917b953a60afd76d2b0b99ff8a71ec
Successfully built mordred


In [2]:
# import pandas for data wrangling
import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn.metrics import accuracy_score


import matplotlib.pyplot as plt
import seaborn as sns
import random

from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import StratifiedKFold, cross_val_score

from rdkit import Chem
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator
from mordred import Calculator, descriptors
import mordred
# Set the seed
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)


out='/notebooks/NEW_FILTERED/'

In [156]:
data_path='path_to/data/your_data'
df = pd.read_csv(data_path,encoding='unicode_escape')
df

,Name,Smile,total,H,P,D
0,PROPANE,CCC,13.10,0.00,0.0,13.100000
1,BUTANE,CCCC,14.10,0.00,0.0,14.100000
2,PENTANE,CCCCC,14.40,0.00,0.0,14.400000
3,HEXANE,CCCCCC,14.90,0.00,0.0,14.900000
4,HEPTANE,CCCCCCC,15.20,0.00,0.0,15.200000
...,...,...,...,...,...,...
56,octanoic acid,CCCCCCCC(=O)O,22.24,8.20,3.3,20.408028
57,oleic acid,CCCCCCCC/C=C\CCCCCCCC(=O)O,17.38,5.50,3.1,16.192727
58,stearic acid,CCCCCCCCCCCCCCCCCC(=O)O,19.04,5.50,3.3,17.927119
59,ammonia,N,24.63,17.80,15.7,6.580798


Calculate the descriptors:

In [147]:

smiles=df['Smile'].tolist()
mols = [Chem.MolFromSmiles(smile) for smile in smiles]

calc = Calculator(descriptors,ignore_3D=True) 
descs = calc.pandas(mols)

descs

100%|██████████| 59/59 [00:01<00:00, 33.02it/s]


,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,1.414214,1.414214,0,0,2.828427,1.414214,2.828427,2.828427,0.942809,1.849457,...,4.174387,17.310771,44.062600,4.005691,4,0,6.0,4.0,2.250000,1.000000
1,2.121320,2.340100,0,0,4.472136,1.618034,3.236068,4.472136,1.118034,2.155909,...,5.509388,22.328143,58.078250,4.148446,10,1,10.0,8.0,2.500000,1.250000
2,2.828427,3.146264,0,0,5.464102,1.732051,3.464102,5.464102,1.092820,2.390167,...,6.192362,25.583106,72.093900,4.240818,20,2,14.0,12.0,2.750000,1.500000
3,3.535534,3.869735,0,0,6.987918,1.801938,3.603875,6.987918,1.164653,2.579830,...,6.608001,28.105124,86.109550,4.305478,35,3,18.0,16.0,3.000000,1.750000
4,4.242641,4.530950,0,0,8.054679,1.847759,3.695518,8.054679,1.150668,2.739193,...,6.900731,30.257210,100.125201,4.353270,56,4,22.0,20.0,3.250000,2.000000
5,4.949747,5.143137,0,0,9.517541,1.879385,3.758770,9.517541,1.189693,2.876615,...,7.126891,32.187603,114.140851,4.390033,84,5,26.0,24.0,3.500000,2.250000
6,5.656854,5.715477,0,0,10.627503,1.902113,3.804226,10.627503,1.180834,2.997414,...,7.311218,33.969149,128.156501,4.419190,120,6,30.0,28.0,3.750000,2.500000
7,6.363961,6.254644,0,0,12.053348,1.918986,3.837972,12.053348,1.205335,3.105180,...,7.466799,35.643477,142.172151,4.442880,165,7,34.0,32.0,4.000000,2.750000
8,7.071068,6.765664,0,0,13.191508,1.931852,3.863703,13.191508,1.199228,3.202455,...,7.601402,37.236738,156.187801,4.462509,220,8,38.0,36.0,4.250000,3.000000
9,7.778175,7.252425,0,0,14.592460,1.941884,3.883767,14.592460,1.216038,3.291100,...,7.720018,38.766486,170.203451,4.479038,286,9,42.0,40.0,4.500000,3.250000


Choose the model (trained on 10, 30 or 50 features), select the descriptors

In [158]:
nfeats=30
with open(f"/trained_models/xgboost/{nfeats}/D_{nfeats}.txt") as file:
    D_features = [line.rstrip() for line in file]
with open(f"trained_models/xgboost/{nfeats}/P{nfeats}.txt") as file:
    P_features = [line.rstrip() for line in file]
with open(f"trained_models/xgboost/{nfeats}/H{nfeats}.txt") as file:
    H_features = [line.rstrip() for line in file]

In [159]:
D_descs=descs[D_features]
P_descs=descs[P_features]
H_descs=descs[H_features]

In [160]:

D = f'/trained_models/xgboost/{nfeats}/MODEL_D_XGBOOST.json'
P = f'/trained_models/xgboost/{nfeats}/MODEL_P_XGBOOST.json'
H = f'/trained_models/xgboost/{nfeats}/MODEL_H_XGBOOST.json'

D_model = xgb.XGBRegressor(enable_categorical=True) #iz nekog razloga ovo treba. iako sam izbacila redove sa greskama i sve
D_model.load_model(D)

P_model = xgb.XGBRegressor(enable_categorical=True)
P_model.load_model(P)

H_model = xgb.XGBRegressor(enable_categorical=True)
H_model.load_model(H)

Just in case,check for errors and remove molecules if they have any errors for their descriptors

In [161]:
ERRS = [mordred.error.MissingValueBase,
        mordred.error.Missing,
        mordred.error.Error,
        mordred.error.MultipleFragments,
        mordred.error.Missing3DCoordinate,
        mordred.error.Timeout]


In [ ]:
# Function to check if a cell contains an error of specified types
def contains_error(cell):
    return any(isinstance(cell, error_type) for error_type in ERRS)

def errors(dataframe):
    # Iterate over rows and filter out rows with errors
    filtered_rows = []
    err_rows = []
    for index, row in dataframe.iterrows():
        if not any(row.apply(contains_error)):
            filtered_rows.append(row)
        else:
            err_rows.append(index)
            print(index)

    return err_rows
errs = list(set(errors(D_descs)+errors(P_descs)+errors(H_descs)))
print(errs)            
D_descs = D_descs.drop(errs)
P_descs = P_descs.drop(errs)
H_descs = H_descs.drop(errs)
D_descs

For some reason, some descriptors are not returned as float, even though they appear to be. If that is the case, then the code wont work - so you have to do this:

In [163]:
from pandas.api.types import is_object_dtype

for col in D_descs.columns:
    if is_object_dtype(D_descs[col]):
        D_descs[col]=D_descs[col].astype(float)
        
for col in P_descs.columns:
    if is_object_dtype(P_descs[col]):
        P_descs[col]=P_descs[col].astype(float)
        
for col in H_descs.columns:
    if is_object_dtype(H_descs[col]):
        H_descs[col]=H_descs[col].astype(float)

In [164]:
df.drop(errs,inplace=True)

y_D = D_model.predict(D_descs)
y_P = P_model.predict(P_descs)
y_H = H_model.predict(H_descs)
#za neparcijalne
df['D predicted'] = y_D
df['P predicted'] = y_P
df['H predicted'] = y_H

df['PREDICTED SOL^2'] = y_D**2+y_P**2+y_H**2
df['PREDICTED SOL'] = np.sqrt(df['PREDICTED SOL^2'])
df

,Name,Smile,total,H,P,D,D predicted,P predicted,H predicted,PREDICTED SOL^2,PREDICTED SOL
0,PROPANE,CCC,13.10,0.00,0.00,13.100000,16.665354,0.473344,0.412899,278.128540,16.677187
1,BUTANE,CCCC,14.10,0.00,0.00,14.100000,16.864727,0.473344,0.167760,284.671204,16.872202
2,PENTANE,CCCCC,14.40,0.00,0.00,14.400000,16.678400,0.473344,0.235927,278.448761,16.686785
3,HEXANE,CCCCCC,14.90,0.00,0.00,14.900000,15.770603,0.473344,0.136321,248.954575,15.778295
4,HEPTANE,CCCCCCC,15.20,0.00,0.00,15.200000,15.714711,0.473344,0.067144,247.180710,15.721982
5,octane,CCCCCCCC,15.40,0.00,0.00,15.400000,15.714711,0.473344,-0.081271,247.182816,15.722049
6,NONANE,CCCCCCCCC,15.60,0.00,0.00,15.600000,15.827431,0.473344,-0.081271,250.738235,15.834716
7,DECANE,CCCCCCCCCC,15.70,0.00,0.00,15.700000,15.827431,0.473344,-0.081271,250.738235,15.834716
8,Undecane,CCCCCCCCCCC,15.80,0.00,0.00,15.800000,15.827431,0.473344,-0.081271,250.738235,15.834716
9,DODECANE,CCCCCCCCCCCC,15.90,0.00,0.00,15.900000,15.827431,0.473344,-0.100542,250.741730,15.834826


In [165]:
#Save the prediction data
df.to_csv('path_to_save_folder/GNNpredictions.csv')